In [2]:
import pandas as pd
import nltk
from nltk.util import ngrams
from collections import Counter

In [3]:
reviews = pd.read_csv('sushi_reviews.csv')

In [5]:
reviews

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,text,date,V10
0,1,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3,0,0,0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06,party 6 hibachi waitress brought separa...
1,2,eCiWBf1CJ0Zdv1uVarEhhw,OhECKhQEexFypOMY6kypRw,vC2qm1y3Au5czBtbhc-DNw,4,0,0,0,"Yes, this is the only sushi place in town. How...",2013-09-04 03:48:20,yes sushi place town however great you...
2,3,lUUhg8ltDsUZ9h0xnwY4Dg,RreNy--tOmXMl1en0wiBOg,cPepkJeRMtHapc_b2Oe_dw,4,1,0,1,I was really between 3 and 4 stars for this on...,2018-07-17 03:30:07,really 3 4 stars one love 96th street ...
3,4,ymhbOMW63B_vGaRFR3XT0A,yZdAhQ_KmKuCZmbBy9YDQg,5Ce3lZksYVkCbrihqylVHQ,5,0,0,0,I just started going to Sushi Nara this month ...,2014-07-25 17:56:26,just started going sushi nara month opened...
4,5,RMho6HMpdec1YgIypwWQrQ,SNngOVGTkD34B3FAFnMv5A,ab3pRv-b0o-BwMK2jVbH3Q,5,0,0,0,My husband and I come here often. We love the ...,2018-08-24 00:52:13,husband come often love food always fr...
...,...,...,...,...,...,...,...,...,...,...,...
215550,215551,3meLluIIU_TFZS1IP9-DcA,6He7CwCnjxeCN9Cs-X8i7g,3FPi2yKCIh0Hh3iBg8faYA,1,5,1,0,"I used to like coming to this restaurant, but ...",2019-02-28 02:10:45,used like coming restaurant last four ti...
215551,215552,2ANk7BzQeUxvC3-sPmkd0w,3oj2pnNtv_eTD-42WGdRFQ,zSSMmKQujl731_voEHStmg,5,0,0,0,I am very happy to have this restaurant in my ...,2015-02-01 21:41:28,happy restaurant neighborhood discov...
215552,215553,FeghE6yk5hmgGFYYqkr5-Q,HiRGRdbSbR7btUmWuZ4_IQ,-5UQGwUZWUhPi5pJlcwEYA,4,2,0,0,"The box lunch is my favorite take out lunch, a...",2016-11-05 20:28:20,box lunch favorite take lunch reasonab...
215553,215554,Wrt6pZizQzw-ZTKrvMwrGw,ua6QuBe6mar6pDrhHETzJQ,Bk_1vsPtOtO0bojfQZQIOw,2,3,3,2,"Ok, I have to say, after living on this block ...",2008-12-15 23:18:27,ok say living block 4 months passing ...


In [18]:
%%time
bag = reviews['V10'].tolist()
bag = ' '.join(str(e) for e in bag)

Wall time: 17 ms


In [37]:
%%time
bigram = list(ngrams(nltk.word_tokenize(bag), 2))  #gen bigrams from bag
temp = dict(Counter(bigram))

df = pd.DataFrame(list(temp.items()))

df.columns = ['Bi-Gram', 'Frequency']
df = df.loc[df.Frequency > 5] #filter more frequent grams 5 mentions or more
df = df.sort_values(by=['Frequency'],ascending=False)
df.to_csv('bigrams_all_reviews.csv')

Wall time: 53.6 s


In [38]:
%%time
trigram = list(ngrams(nltk.word_tokenize(bag), 3))  #gen trigrams from bag
temp = dict(Counter(trigram))

df = pd.DataFrame(list(temp.items()))

df.columns = ['Tri-Gram', 'Frequency']
df = df.loc[df.Frequency > 5] #filter more frequent grams 5 mentions or more
df = df.sort_values(by=['Frequency'],ascending=False)
df.to_csv('trigrams_all_reviews.csv')

Wall time: 57.4 s


In [51]:
df_uni = pd.read_csv('unigrams_all_reviews.csv')
df_bi = pd.read_csv('bigrams_all_reviews.csv')
df_tri = pd.read_csv('trigrams_all_reviews.csv')


In [50]:
all_grams = pd.concat([df_uni, df_bi, df_tri], axis=1)
all_grams = all_grams.drop(['Unnamed: 0'], axis=1)

all_grams.to_csv('all_grams_all_reviews.csv', index = False)

# Filter Reviews by star and make n-gram tables 


In [79]:
%%time
#first loop for filter new dataset
for i in range(1,6): #stars
    current_star = f'{i}-Star'
    temp_df = reviews.loc[reviews.stars == i]
    
    bag = ''
    bag = temp_df['V10'].tolist()
    bag = ' '.join(str(e) for e in bag)

    #second loop for 1,2,3 grams
    for j in range(1,4):
        current_gram = f'{j}-Gram'
        grams = list(ngrams(nltk.word_tokenize(bag), j))  #gen j-Grams
        temp = dict(Counter(grams))
        if j == 1:
            gram_df = pd.DataFrame(list(temp.items()))
            gram_df.columns = [current_gram, 'Frequency']
            gram_df = gram_df.loc[gram_df.Frequency > 5] #filter more frequent grams 5 mentions or more
            gram_df = gram_df.sort_values(by=['Frequency'],ascending=False)
            
        else:
            temp = pd.DataFrame(list(temp.items()))
            temp.columns = [current_gram, 'Frequency']
            temp = temp.loc[temp.Frequency > 5] #filter more frequent grams 5 mentions or more
            temp = temp.sort_values(by=['Frequency'],ascending=False)
            temp.reset_index(drop=True, inplace=True)  #fix bug where indices cause ordering issue in df
            gram_df.reset_index(drop=True, inplace=True)  #fix bug where indices cause ordering issue in df
            gram_df = pd.concat([gram_df, temp], axis=1)
    gram_df.to_csv(f'{current_star}-grams.csv')

Wall time: 2min 47s


In [65]:
all_grams.head(20)

,Uni-Gram,Frequency,Bi-Gram,Frequency,Tri-Gram,Frequency
0,"('sushi',)",231481.0,"('can', 'eat')",10717,"('can', 'eat', 'sushi')",3255.0
1,"('food',)",134406.0,"('sushi', 'place')",10704,"('will', 'definitely', 'back')",2996.0
2,"('good',)",124300.0,"('best', 'sushi')",10578,"('spicy', 'tuna', 'roll')",2989.0
3,"('place',)",119781.0,"('first', 'time')",10335,"('favorite', 'sushi', 'place')",1687.0
4,"('great',)",98003.0,"('go', 'back')",9754,"('wait', 'go', 'back')",1537.0
5,"('service',)",84745.0,"('happy', 'hour')",9643,"('best', 'sushi', 'ive')",1499.0
6,"('roll',)",84728.0,"('miso', 'soup')",9493,"('definitely', 'come', 'back')",1487.0
7,"('rolls',)",72171.0,"('spicy', 'tuna')",9078,"('cant', 'wait', 'go')",1475.0
8,"('like',)",67517.0,"('sushi', 'rolls')",8967,"('food', 'great', 'service')",1181.0
9,"('just',)",65253.0,"('come', 'back')",8851,"('definitely', 'go', 'back')",1176.0


In [81]:
pd.read_csv('1-Star-grams.csv').head(20)

,Unnamed: 0,1-Gram,Frequency,2-Gram,Frequency.1,3-Gram,Frequency.2
0,0,"('food',)",21083.0,"('will', 'never')",1975,"('never', 'go', 'back')",570.0
1,1,"('sushi',)",20366.0,"('customer', 'service')",1579,"('will', 'never', 'go')",547.0
2,2,"('place',)",13275.0,"('go', 'back')",1353,"('can', 'eat', 'sushi')",319.0
3,3,"('us',)",12849.0,"('first', 'time')",1227,"('will', 'never', 'return')",270.0
4,4,"('service',)",11440.0,"('can', 'eat')",1198,"('never', 'come', 'back')",247.0
5,5,"('order',)",10784.0,"('tasted', 'like')",1085,"('spicy', 'tuna', 'roll')",240.0
6,6,"('just',)",10604.0,"('didnt', 'even')",1001,"('dont', 'waste', 'time')",212.0
7,7,"('like',)",9893.0,"('come', 'back')",992,"('never', 'going', 'back')",212.0
8,8,"('one',)",9811.0,"('20', 'minutes')",953,"('worst', 'service', 'ever')",209.0
9,9,"('back',)",9618.0,"('fried', 'rice')",953,"('will', 'never', 'come')",205.0


In [82]:
pd.read_csv('2-Star-grams.csv').head(20)

,Unnamed: 0,1-Gram,Frequency,2-Gram,Frequency.1,3-Gram,Frequency.2
0,0,"('sushi',)",21115.0,"('go', 'back')",1061,"('spicy', 'tuna', 'roll')",328.0
1,1,"('food',)",15983.0,"('miso', 'soup')",963,"('can', 'eat', 'sushi')",260.0
2,2,"('good',)",11719.0,"('first', 'time')",944,"('wont', 'going', 'back')",159.0
3,3,"('place',)",10568.0,"('can', 'eat')",933,"('food', 'good', 'service')",136.0
4,4,"('just',)",9936.0,"('food', 'good')",913,"('will', 'go', 'back')",129.0
5,5,"('like',)",9137.0,"('tasted', 'like')",872,"('will', 'going', 'back')",123.0
6,6,"('service',)",9032.0,"('fried', 'rice')",870,"('really', 'wanted', 'like')",120.0
7,7,"('ordered',)",7782.0,"('sushi', 'rolls')",817,"('dont', 'think', 'ill')",116.0
8,8,"('roll',)",7316.0,"('spicy', 'tuna')",803,"('wont', 'coming', 'back')",115.0
9,9,"('one',)",7300.0,"('come', 'back')",771,"('food', 'pretty', 'good')",114.0


In [83]:
pd.read_csv('3-Star-grams.csv').head(20)

,Unnamed: 0,1-Gram,Frequency,2-Gram,Frequency.1,3-Gram,Frequency.2
0,0,"('sushi',)",29421.0,"('pretty', 'good')",2164,"('spicy', 'tuna', 'roll')",461.0
1,1,"('good',)",21790.0,"('go', 'back')",1619,"('can', 'eat', 'sushi')",391.0
2,2,"('food',)",18373.0,"('food', 'good')",1544,"('food', 'pretty', 'good')",230.0
3,3,"('place',)",14168.0,"('happy', 'hour')",1480,"('sushi', 'pretty', 'good')",193.0
4,4,"('just',)",11949.0,"('miso', 'soup')",1414,"('food', 'good', 'service')",175.0
5,5,"('like',)",11470.0,"('can', 'eat')",1300,"('will', 'go', 'back')",171.0
6,6,"('roll',)",11253.0,"('come', 'back')",1294,"('give', 'another', 'try')",165.0
7,7,"('service',)",11197.0,"('spicy', 'tuna')",1280,"('nothing', 'write', 'home')",156.0
8,8,"('rolls',)",10643.0,"('first', 'time')",1227,"('shrimp', 'tempura', 'roll')",152.0
9,9,"('time',)",8482.0,"('sushi', 'rolls')",1207,"('go', 'back', 'try')",146.0


In [84]:
pd.read_csv('4-Star-grams.csv').head(20)

,Unnamed: 0,1-Gram,Frequency,2-Gram,Frequency.1,3-Gram,Frequency.2
0,0,"('sushi',)",60710.0,"('happy', 'hour')",3323,"('can', 'eat', 'sushi')",974.0
1,1,"('good',)",43021.0,"('really', 'good')",3261,"('spicy', 'tuna', 'roll')",919.0
2,2,"('food',)",29292.0,"('pretty', 'good')",3234,"('will', 'definitely', 'back')",746.0
3,3,"('place',)",28762.0,"('can', 'eat')",3047,"('definitely', 'go', 'back')",546.0
4,4,"('great',)",27962.0,"('spicy', 'tuna')",2913,"('definitely', 'come', 'back')",545.0
5,5,"('roll',)",25303.0,"('miso', 'soup')",2899,"('roll', 'spicy', 'tuna')",358.0
6,6,"('rolls',)",21890.0,"('sushi', 'rolls')",2634,"('shrimp', 'tempura', 'roll')",348.0
7,7,"('service',)",19136.0,"('go', 'back')",2513,"('soft', 'shell', 'crab')",336.0
8,8,"('like',)",17513.0,"('sushi', 'place')",2495,"('green', 'tea', 'ice')",323.0
9,9,"('really',)",16658.0,"('first', 'time')",2485,"('come', 'back', 'try')",311.0


In [85]:
pd.read_csv('5-Star-grams.csv').head(20)

,Unnamed: 0,1-Gram,Frequency,2-Gram,Frequency.1,3-Gram,Frequency.2
0,0,"('sushi',)",99869.0,"('best', 'sushi')",8265,"('will', 'definitely', 'back')",2210.0
1,1,"('great',)",54508.0,"('highly', 'recommend')",6274,"('can', 'eat', 'sushi')",1311.0
2,2,"('place',)",53008.0,"('sushi', 'place')",5719,"('wait', 'go', 'back')",1293.0
3,3,"('food',)",49675.0,"('will', 'definitely')",5688,"('favorite', 'sushi', 'place')",1282.0
4,4,"('good',)",39773.0,"('great', 'service')",4470,"('cant', 'wait', 'go')",1231.0
5,5,"('roll',)",34633.0,"('first', 'time')",4452,"('best', 'sushi', 'ive')",1090.0
6,6,"('service',)",33940.0,"('can', 'eat')",4239,"('spicy', 'tuna', 'roll')",1041.0
7,7,"('fresh',)",27274.0,"('love', 'place')",4137,"('highly', 'recommend', 'place')",924.0
8,8,"('best',)",27056.0,"('great', 'food')",3816,"('definitely', 'come', 'back')",848.0
9,9,"('rolls',)",26884.0,"('service', 'great')",3659,"('definitely', 'coming', 'back')",827.0
